In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
movies=pd.read_csv('Data/movies.csv')
credits=pd.read_csv('Data/credits.csv')
# here we got dataframes and it will cause trouble so lets merge them into a single dataframe

In [3]:
movies = movies.merge(credits, on='title')
movies.info()
# we don't need all these columns for our model 
# preparation so we will select a few columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [4]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.6+ KB


In [5]:
#checking for null columns
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [6]:
movies.dropna(inplace=True)
#dropping the rows with null coulumns

In [7]:
#checking for duplicates
movies.duplicated().sum()

0

In [8]:
movies.iloc[0].genres

#we need to convert dictionary of genres to list  so........

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l
    
#function to fetch directors name from crew
def fetch_director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

#function to pick top 3 actors
def convert3(obj):
    l=[]
    c=0
    for i in ast.literal_eval(obj):
        if(c!=3):
            l.append(i['name'])
            c+=1
        else:
            break
    return l

In [10]:
movies['genres']=movies['genres'].apply(convert)


In [11]:
movies['keywords']=movies['keywords'].apply(convert)


In [12]:
movies['cast']=movies['cast'].apply(convert3)



In [13]:
movies['crew']=movies['crew'].apply(fetch_director)


In [14]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [15]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [16]:
#spaces between first names and last names of actors can decrease model accuracy
# because for model Tom Cruise is divided in to 2 entities like #tom & #cruise not that tom
# will also force our model to recommend movies every Tom like Tom Hiddleston, Tom Holland..
# so we will make Tom cruise a sinlge entity by removing space between first and ast name #TomCruise


In [17]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [18]:
movies['tags']=movies['cast'] + movies['crew'] + movies['genres'] + movies ['keywords'] + movies['overview']

In [19]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[DanielCraig, ChristophWaltz, LéaSeydoux, SamM..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[ChristianBale, MichaelCaine, GaryOldman, Chri..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[TaylorKitsch, LynnCollins, SamanthaMorton, An..."


In [20]:
#now i don't need all columns so lets proceed
newDf = movies[['movie_id', 'title', 'tags']]


In [21]:
newDf['tags'] = newDf['tags'].apply(lambda x:" ".join(x))
newDf['tags'] = newDf['tags'].apply(lambda x:x.lower())

/tmp/ipykernel_2971/1592558264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags'] = newDf['tags'].apply(lambda x:" ".join(x))
/tmp/ipykernel_2971/1592558264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags'] = newDf['tags'].apply(lambda x:x.lower())


In [22]:
newDf.head()

,movie_id,title,tags
0,19995,Avatar,samworthington zoesaldana sigourneyweaver jame...
1,285,Pirates of the Caribbean: At World's End,johnnydepp orlandobloom keiraknightley gorever...
2,206647,Spectre,danielcraig christophwaltz léaseydoux sammende...
3,49026,The Dark Knight Rises,christianbale michaelcaine garyoldman christop...
4,49529,John Carter,taylorkitsch lynncollins samanthamorton andrew...


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000, stop_words='english')
# stop words are - is, am , are, in, to , on, the, from , for, here, etc..
# so we remove english stop words because stop words don't play any 
# role in machine Learning Model

In [27]:
vectors=vectorizer.fit_transform(newDf['tags']).toarray()
#this vectors variable conains those 5000 words ... have high frequency in 5000 movies title, overview, tags, actor, cast, etc
# responsible for fitting the transformer and then return the transformed training instances
# by default it will return Scipy Sparse Matrix theefore we convert it to numpy array
#sparse matrix has most of the elements 0 

In [64]:
x=vectorizer.get_feature_names()

#there are words like action, actions, actor actors, love , loves, loving
# so instead of recongnizing them 7 different words we have to  do somethinf so our model
# recognize them as only 3 different words
# love, action and actor
# to overcome this problem we gonna use stemming from NLTK library

/home/sachin108/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [37]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [41]:
def stemming(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [42]:
newDf['tags']=newDf['tags'].apply(stemming)

/tmp/ipykernel_2971/821839388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags']=newDf['tags'].apply(stemming)


In [44]:
newDf['tags'][0]

# the output is stem

'samworthington zoesaldana sigourneyweav jamescameron action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization.'

In [47]:
newDf['tags'].shape
 # i.e. we have 4806 movies and each movie's vector has 5000 element in 
 # form of numbers determining
 #frequency of that word in that particular movie

(4806,)

In [48]:
'''each movie is vector in multi-dimensional imaginary space and now we havr to calculate
the distance between each vector in that space ... we can calculate it using Euclidean diatance but.. 
we will use  cosine distance :) for better accuracy
cosine distance is angle between 2 vectors.T
Cosine similarity is a metric, helpful in determining, how similar
 the data objects are irrespective of their size
 '''

from sklearn.metrics.pairwise import cosine_similarity

In [49]:
cosine_similarity(vectors)

# the distance of each movie has been calculated with every other movie

array([[1.        , 0.08964215, 0.05976143, ..., 0.02519763, 0.02817181,
        0.        ],
       [0.08964215, 1.        , 0.0625    , ..., 0.02635231, 0.        ,
        0.        ],
       [0.05976143, 0.0625    , 1.        , ..., 0.02635231, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02635231, ..., 1.        , 0.0745356 ,
        0.04836508],
       [0.02817181, 0.        , 0.        , ..., 0.0745356 , 1.        ,
        0.05407381],
       [0.        , 0.        , 0.        , ..., 0.04836508, 0.05407381,
        1.        ]])

In [52]:
similarity = cosine_similarity(vectors)

In [54]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

'''
we enumerate all the movied because we have to hold their indices,
 and then we sort the list and fetch first five or six movies ... bcz after sorting these movies 
 must have cosine similarity approximate equal to that particular searched movie
'''

[(539, 0.26089696604360174),
 (1194, 0.2581988897471611),
 (507, 0.25302403842552984),
 (260, 0.25110592822973776),
 (1216, 0.24944382578492943)]

In [62]:
def recommend(movie):
    movie_index = newDf[newDf['title']==movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    for i in  movie_list:
        print(newDf.iloc[i[0]].title)


In [63]:
recommend("Avatar")

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem
